#### Import Libraries

In [1]:
import os
import folium
import numpy as np
import pandas as pd
import geopandas as gpd
import requests
print(f'Folium Version: {folium.__version__}')

Folium Version: 0.14.0


#### Get VWorld Tiles

##### Base Settings

In [2]:
BASE_KEY_FILE_PATH = os.path.join(
    os.path.expanduser('~'),
    'projects' if os.name == 'posix' else 'python_projects'
)

VWORLD_KEY_FILE_PATH = os.path.join(BASE_KEY_FILE_PATH, 'vworld_key.txt')
with open(VWORLD_KEY_FILE_PATH, 'r') as vkf:
    vk_lines = vkf.readlines()
VWORLD_KEY = vk_lines[0].replace('\n', '')

In [3]:
VWORLD_TILE_TYPE='png'
VWORLD_ATTR = 'Vworld'

##### Tile: Base(기본도)

In [4]:
VWORLD_LAYER = 'Base'
VWORLD_TILES_URL = \
    f'http://api.vworld.kr/req/wmts/1.0.0/' \
    f'{VWORLD_KEY}/{VWORLD_LAYER}/{{z}}/{{y}}/{{x}}.{VWORLD_TILE_TYPE}'

base_map = folium.Map(
    location=[37.5665, 126.9780], zoom_start=13,
    tiles=VWORLD_TILES_URL, attr=VWORLD_ATTR
)
base_map

##### Tile: Hybrid(하이브리드지도)

In [7]:
VWORLD_LAYER = 'Hybrid'
VWORLD_TILES_URL = \
    f'http://api.vworld.kr/req/wmts/1.0.0/' \
    f'{VWORLD_KEY}/{VWORLD_LAYER}/{{z}}/{{y}}/{{x}}.{VWORLD_TILE_TYPE}'

base_map = folium.Map(
    location=[37.5665, 126.9780], zoom_start=13,
    tiles=VWORLD_TILES_URL, attr=VWORLD_ATTR
)
base_map

##### Tiles: midnight(어두운 배경)

In [8]:
VWORLD_LAYER = 'midnight'
VWORLD_TILES_URL = \
    f'http://api.vworld.kr/req/wmts/1.0.0/' \
    f'{VWORLD_KEY}/{VWORLD_LAYER}/{{z}}/{{y}}/{{x}}.{VWORLD_TILE_TYPE}'

base_map = folium.Map(
    location=[37.5665, 126.9780], zoom_start=13,
    tiles=VWORLD_TILES_URL, attr=VWORLD_ATTR
)
base_map

##### Tile: white(밝은 배경)

In [9]:
VWORLD_LAYER = 'white'
VWORLD_TILES_URL = \
    f'http://api.vworld.kr/req/wmts/1.0.0/' \
    f'{VWORLD_KEY}/{VWORLD_LAYER}/{{z}}/{{y}}/{{x}}.{VWORLD_TILE_TYPE}'

base_map = folium.Map(
    location=[37.5665, 126.9780], zoom_start=13,
    tiles=VWORLD_TILES_URL, attr=VWORLD_ATTR
)
base_map

#### 행정구역별 구분

In [10]:
# SHP PATH
VW_SIGNGU_PATH = os.path.join(
    os.getcwd(), '89_data', 'vw-lab.HangJeongDong_ver20220701.geojson'
)

# 파일읽는 동안 
# DriverError: .. supported file format
# 관련 오류가 발생하면 gdal이 시스템에 설치 안되 그런것이지 아래와 같이 설치
# > sudo apt install gdal-bin python3-gdal
signgu_gdf = gpd.read_file(VW_SIGNGU_PATH)
signgu_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 3513 entries, 0 to 3512
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   OBJECTID  3513 non-null   int64   
 1   adm_nm    3513 non-null   object  
 2   adm_cd    3513 non-null   object  
 3   adm_cd2   3513 non-null   object  
 4   sgg       3513 non-null   object  
 5   sido      3513 non-null   object  
 6   sidonm    3513 non-null   object  
 7   sggnm     3513 non-null   object  
 8   adm_cd8   3513 non-null   object  
 9   geometry  3513 non-null   geometry
dtypes: geometry(1), int64(1), object(8)
memory usage: 274.6+ KB


In [11]:
signgu_gdf.head()

,OBJECTID,adm_nm,adm_cd,adm_cd2,sgg,sido,sidonm,sggnm,adm_cd8,geometry
0,1,서울특별시 종로구 사직동,1101053,1111053000,11110,11,서울특별시,종로구,11010530,"MULTIPOLYGON (((126.97689 37.57565, 126.97703 ..."
1,2,서울특별시 종로구 삼청동,1101054,1111054000,11110,11,서울특별시,종로구,11010540,"MULTIPOLYGON (((126.98269 37.59507, 126.98337 ..."
2,3,서울특별시 종로구 부암동,1101055,1111055000,11110,11,서울특별시,종로구,11010550,"MULTIPOLYGON (((126.97585 37.59656, 126.97359 ..."
3,4,서울특별시 종로구 평창동,1101056,1111056000,11110,11,서울특별시,종로구,11010560,"MULTIPOLYGON (((126.97507 37.63139, 126.97649 ..."
4,5,서울특별시 종로구 무악동,1101057,1111057000,11110,11,서울특별시,종로구,11010570,"MULTIPOLYGON (((126.96067 37.58080, 126.96281 ..."


In [12]:
# working_signgu_gdf
ws_gdf = signgu_gdf.copy()

In [13]:
# 중앙값 계산이 용이한 좌표계(4326->5179)로 변환
ws_gdf = ws_gdf.to_crs(epsg=5179)

ws_gdf['center'] = ws_gdf.centroid

##### 나주시 추출

In [14]:
naju_gdf = ws_gdf[signgu_gdf.adm_nm.str.contains('빛가람')]

In [15]:
naju_gdf = naju_gdf.drop(columns=['geometry'])

In [16]:
naju_gdf = naju_gdf.rename(columns={'center': 'geometry'})

In [17]:
naju_gdf = naju_gdf.to_crs(epsg=4326)
naju_gdf.head()

,OBJECTID,adm_nm,adm_cd,adm_cd2,sgg,sido,sidonm,sggnm,adm_cd8,geometry
2531,2555,전라남도 나주시 빛가람동,3604061,4617062000,46170,46,전라남도,나주시,36040610,POINT (126.78909 35.01830)


In [18]:
center = [naju_gdf.iloc[0].geometry.y, naju_gdf.iloc[0].geometry.x]

#### 빛가람동 기본맵 표시

In [19]:
_center = [35.0098072457794, 126.79208767982794]
VWORLD_LAYER = 'Base'
VWORLD_TILES_URL = \
    f'http://api.vworld.kr/req/wmts/1.0.0/' \
    f'{VWORLD_KEY}/{VWORLD_LAYER}/{{z}}/{{y}}/{{x}}.{VWORLD_TILE_TYPE}'

base_map = folium.Map(
    location=_center, zoom_start=17,
    tiles=VWORLD_TILES_URL, attr=VWORLD_ATTR
)

folium.Marker(
    location=_center,
    popup='Start',
    icon=folium.Icon(color='blue')
).add_to(base_map)

print(_center)
base_map

[35.0098072457794, 126.79208767982794]


#### 네이버 경로분석

##### 네이버 길찿기 함수 생성

In [21]:
NAVER_KEY_PATH = os.path.join(BASE_KEY_FILE_PATH, 'naver_application_key.txt')

with open(NAVER_KEY_PATH, 'r') as nkf:
    lines = nkf.readlines()

NAVER_REQ_HEADER = {
    "X-NCP-APIGW-API-KEY-ID": lines[0].replace('\n', ''),
    "X-NCP-APIGW-API-KEY": lines[1].replace('\n', '')
}

In [22]:
def get_naver_path(start, end, way):
    optimal_api_url = \
        f'https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving?' \
        f'start={start[1]},{start[0]}&goal={end[1]},{end[0]}' \
        f'&waypoints={way[1]},{way[0]}&option=trafast&format=json'
        
    response = requests.get(optimal_api_url, headers=NAVER_REQ_HEADER)
    return response.json() if response.status_code == 200 else {'status': 'error'}

In [34]:
def get_geocoding(addr):
    geocoding_url = f'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query={addr}'
    response = requests.get(geocoding_url, headers=NAVER_REQ_HEADER)
    
    try:
        latlon = response.json()['addresses'][0]
        return [float(latlon['y']), float(latlon['x'])]
    except IndexError:
        print(f'Addr("{addr}") not found.') 

##### 시작-경유지-종료지점 좌표 설정

In [20]:
SP_ADDR = '전라남도 나주시 남고문로 104'
M1_ADDR = '전라남도 나주시 학생운동길 21-1'
M2_ADDR = '전라남도 나주시 남고문로 108-10'
M3_ADDR = '전라남도 나주시 남고문로 118'
EP_ADDR = '전라남도 나주시 학생운동길 36'
# SP = [35.0289538, 126.7221748]          # 남고문로 104
# M1 = [35.0282973, 126.7221589]          # 학생운동길 21-1
# M2 = [35.0286806, 126.7230117]          # 남고문로 108-10
# M3 = [35.0287562, 126.7236250]          # 남고문로 118
# EP = [35.0276670, 126.7234553]          # 학생운동길 36

In [40]:
SP = get_geocoding(SP_ADDR)
EP = get_geocoding(EP_ADDR)
M1 = get_geocoding(M1_ADDR)
M2 = get_geocoding(M2_ADDR)
M3 = get_geocoding(M3_ADDR)

In [41]:
m1_path = get_naver_path(SP, EP, M1)['route']['trafast'][0]['path']
m2_path = get_naver_path(SP, EP, M2)['route']['trafast'][0]['path']
m3_path = get_naver_path(SP, EP, M3)['route']['trafast'][0]['path']

##### 기본지도 생성

In [42]:
VWORLD_LAYER = 'Base'

_center = [(SP[0]+EP[0])/2, (SP[1]+EP[1])/2]
VWORLD_TILES_URL = \
    f'http://api.vworld.kr/req/wmts/1.0.0/' \
    f'{VWORLD_KEY}/{VWORLD_LAYER}/{{z}}/{{y}}/{{x}}.{VWORLD_TILE_TYPE}'

# 기본지도 생성
base_map = folium.Map(
    location=_center, tiles=VWORLD_TILES_URL, attr=VWORLD_ATTR, zoom_start=18
)

# 경로추가
m11_path = [[y, x] for x, y in m1_path]
folium.PolyLine(locations=m11_path, color='green').add_to(base_map)
m22_path = [[y, x] for x, y in m2_path]
folium.PolyLine(locations=m22_path, color='blue').add_to(base_map)
m33_path = [[y, x] for x, y in m3_path]
folium.PolyLine(locations=m33_path, color='red').add_to(base_map)

# 시작/종점 추가
folium.Marker(
    location=SP,
    popup='Start',
    icon=folium.Icon(color='blue')
).add_to(base_map)
folium.Marker(
    location=EP,
    popup='End',
    icon=folium.Icon(color='red')
).add_to(base_map)

# 지도표시
base_map

In [28]:
base_map.save('./89_data/result/52_naju.html')